### estimate plasmid contigs using references

In [80]:
import sys,os,subprocess,glob,shutil
%matplotlib inline
import pylab as plt
import numpy as np
import pandas as pd
pd.set_option('display.width', 110)
pd.set_option('display.max_colwidth', 80)
import seaborn as sns
from IPython.display import display, HTML, Markdown
from Bio import SeqIO, SeqUtils
from dna_features_viewer import BiopythonTranslator
from BCBio import GFF
from epitopepredict import sequtils
from smallrnaseq import utils
from tools import *

In [16]:
info = pd.read_csv('isolates.csv')
names = info.id
dog=['9805','9808']
acinet = ['RF14B','RF15A','RF15B']#,'909D3A']
ecoli = ['RF1A', 'RF2A', 'RF2B', 'RF2C', 'RF5A', 'RF6A1', 'RF6A2', 'RF6B', 'RF6C', 'RF7A', 
         'RF8A', 'RF8B', 'RF9', 'RF11', 'RF12A', 'RF14A', 'RF16A']

### run nucmer against refs so we can exclude non-genomic contigs

In [ ]:
ref='ecoli_all'  
#ref = 'Acin_lwof_ZS207'
#ref = 'Acin_baum_ASM74664'

cpath='contigs'
for n in ref:
    cmd='nucmer --coords -p nucmer genomes/{r}.fa {p}/{n}.fa'.format(p=cpath,r=ref,n=n)    
    #cmd='promer --coords -p promer genomes/{r}.fa {p}/{n}.fa'.format(p=cpath,r=ref,n=n)
    print cmd
    subprocess.check_output(cmd, shell=True)
    shutil.move('nucmer.coords', os.path.join(cpath,'%s_nucmer.coords' %n))

In [27]:
reload(sequtils)

def read_nucmer_coords(cfile):
    cols=['S1','E1','S2','E2','LEN 1','LEN 2','IDENT','TAG1','TAG2']
    a=pd.read_csv(cfile,sep='[\s|]+',skiprows=5,names=cols,engine='python')
    return a

def get_plasmids(name, cutoff=2000):
    #guess plasmid contigs by comparing to reference
    
    path = 'scaffolds'
    df = utils.fasta_to_dataframe(os.path.join(path,'%s.fa' %name)).reset_index()    
    cfile = os.path.join(path,'%s_nucmer.coords' %name)
    a = read_nucmer_coords(cfile)  
    found = a.TAG2    

    pl = df[~df.name.isin(found)]
    pl['length'] = pl.sequence.str.len()
    pl = pl[pl.length>cutoff]
    #print pl    
    return pl


In [ ]:
#a = read_nucmer_coords('contigs_new/RF14B_nucmer.coords')  
#a.sort_values(by='LEN 1',ascending=False)

pl=get_plasmids('9808') 
sequtils.dataframe_to_fasta(pl,idkey='name',seqkey='sequence')
pl

In [ ]:
res=[]
for name in ecoli+dog+acinet:
    #print name
    #display(Markdown('**%s**' %name))
    pl=get_plasmids(name, cutoff=2000)
    pl['isolate'] = name    
    res.append(pl)
    out = 'scaffolds/%s_plasmids.fa' %name
    utils.dataframe_to_fasta(pl,outfile=out,idkey='name',seqkey='sequence')
    #cids=list(pl.name)[:5]
    #gff_file = 'annot_new/{n}/{n}.gff'.format(n=name)    
    #feats = get_features(gff_file, cids, plot=False)
    #print feats
    #print ('-----------------------------------------')
    
plas=pd.concat(res)
plas=plas.set_index('isolate')
plas.to_csv('plasmids_guessed.csv')

In [40]:
def get_contig_seqs(name, contig):
    f = 'scaffolds/%s.fa' %name
    seq = SeqIO.to_dict( SeqIO.parse(f,'fasta') )
    #seqs = dict([(i,seqs[i]) for i in contigs])        
    return seq[contig]
    
def contig_calcs(df):
    df['coverage'] = df.apply(lambda x: x['SEQUENCE'].split('_')[5],1).astype('float')
    df['length'] = df.apply(lambda x: x['SEQUENCE'].split('_')[3],1).astype('float')    
    df['gc'] = df.apply(lambda x: SeqUtils.GC(x.seq),1)
    return df

### contig sequence for plasmids

In [ ]:
i=0
abr = pd.read_csv('abr_plasmidfinder.csv')
plas = []
for n in ecoli:
    conts = sequtils.fasta_to_dataframe('contigs/%s.fa' %n,key='SEQUENCE',seqkey='seq')
    #conts = contig_calcs(conts)
    #print n,len(conts), conts.gc.median()
    cf = abr[abr.id==n].drop_duplicates('SEQUENCE')
    conts = list(cf.SEQUENCE) 
    print n
    #print cf
    for i,r in cf.iterrows():
        c = r.SEQUENCE
        seq = get_contig_seq(n, c)
        seq.id = r.id+'_'+r.GENE
        print seq.id
        plas.append(seq)
SeqIO.write(plas,'plasmid_contigs.fa','fasta')

### plasmid blast results

In [104]:
cols='query,subject,%identity,alignment_length,mismatches,gap opens,qstart, qend,sstart,send,evalue,bit_score'
x = pd.read_csv('plasmid_contig_blast.csv')
x['isolate'] = x['query'].apply(lambda x: x.split('_')[0],1)
hits = x.groupby('query').first().reset_index()

id_list = list(hits.subject)
#ann = retrieve_annotation(id_list)
annot = pd.DataFrame([(i['Title'],i['AccessionVersion']) for i in ann],columns=['title','subject'])
hits = hits.merge(annot,on='subject',how='left').drop_duplicates()
print hits[:10]
hits.to_csv('plasmid_top_hits.csv')

                     query     subject  %identity  alignment_length  mismatches  gap opens  qstart   qend  \
0        RF11_Col(MG828)_1  LM996592.1     99.936              1551           1          0      76   1626   
4            RF11_Col156_1  CP008716.1     96.654               538          18          0       1    538   
5           RF11_Col8282_1  LC056195.1     99.926              4056           3          0      47   4102   
7    RF11_IncFIB(K)_1_Kpn3  CP024976.1     99.658              6430          21          1    3629  10057   
11      RF11_IncFIC(FII)_1  CP024976.1    100.000              2466           0          0     128   2593   
15           RF11_IncFII_1  LC318050.1    100.000              4871           0          0     124   4994   
16      RF11_IncI1_1_Alpha  CP021208.1     99.307             17307         111          2     128  17434   
17         RF12A_ColRNAI_1  CP014198.2     99.605              6581          25          1    6883  13463   
18      RF12A_IncB/

In [ ]:
n='RF14B'
clrs=['b','r','g']
f,ax=plt.subplots(3,1,figsize=(14,5))
ax=ax.flat
    #print SeqUtils.GC(s)
    #print cf[['id','SEQUENCE','GENE']]        
    #conts.hist('gc',bins=35,ax=ax[i],color=clrs[i])
    #conts.plot('coverage','gc',kind='scatter',ax=ax[i],alpha=.4,s=50,c=clrs[i])
    i+=1
    #print conts[conts.gc>70]

In [ ]:
print y[(y.gc<45) & (y.coverage>50)]
print x.coverage.median()
print y.coverage.median()
f,ax=plt.subplots(1,1)
y.hist('gc',bins=25,ax=ax)
x.hist('gc',bins=25,ax=ax.twinx(),color='red',alpha=.5)
f,ax=plt.subplots(1,1)
y.plot('coverage','gc',kind='scatter',figsize=(14,5),ax=ax,alpha=.5,s=10)
x.plot('coverage','gc',kind='scatter',figsize=(14,5),ax=ax,c='red',alpha=.5)
ax.set_xlim(-5,200)

In [ ]:
for name in samples:
    print name
    display(pl.loc[(name)][cols])
    gff_file = 'annot/{n}/{n}.gff'.format(n=name) 
    feats = get_features(gff_file, found, plot=True)

In [ ]:
pl=plas.reset_index().drop('sequence',1)
isf = pd.read_csv('isfinder_results.csv')
print isf[:2]
pl=pl.merge(isf,left_on='name',right_on='contig')
print pl